In [1]:
from collections import deque
import numpy as np
from queue import PriorityQueue
from myheapq import _heapify_max, _heappop_max, _siftdown_max

INFINITY = np.iinfo(np.int32).max

In [2]:
def heap_push(max_heap, item):
    max_heap.append(item)
    _siftdown_max(max_heap, 0, len(max_heap) - 1)

def heap_pop(arr):
    return _heappop_max(arr)  # Converts array to max_heap

In [10]:
class Processor:
    def __init__(self):
        self.task_allocation_queue = deque()
        self.avail = 0

In [4]:
class Function:
    def __init__(self, processors: list, n, criticality, communication_cost, computation_time, edges, arrival_time, deadline, lower_bound):
        self.processors = processors
        self.communication_cost = communication_cost
        self.computation_cost = computation_time
        self.edges = edges
        self.task_priority_queue = []  # <function_number, number, rank_u, criticality>
        self.criticality = criticality
        self.upward_rank = np.zeros(n)
        self.arrival_time = arrival_time
        self.deadline = deadline
        self.lower_bound = lower_bound
        self.task_lower_bound = [(0, 0) for _ in range(n)]
        self.AFT = np.zeros(n)

    def get_EFT(self, task, k):
        return self.get_EST(task, k) + self.computation_cost[task][k]

    def get_EST(self, task, k):
        return max(self.processors[k].avail, self.find_max_AFT_plus_comcost(task))

    def find_max_AFT_plus_comcost(self, task):
        candidate_max = 0
        for pred_task in self.get_predecessor(task):
            if candidate_max < self.AFT[pred_task] + self.communication_cost[pred_task][task]:
                candidate_max = self.AFT[pred_task] + self.communication_cost[pred_task][task]
        return candidate_max

    def get_abs_deadline(self):
        return self.arrival_time + self.deadline

    def get_task_abs_deadline(self, task):
        return self.arrival_time + self.calculate_lower_bound(task)[0] + self.get_deadline_slack()

    def get_deadline_slack(self):
        return self.deadline - self.lower_bound

    def calculate_lower_bound(self, task):
        if self.task_lower_bound[task][0] != 0:
            return self.task_lower_bound[task]
        if task == 0:
            self.task_lower_bound[task] = (np.min(self.computation_cost[task]), np.argmin(self.computation_cost[task]))
            return self.task_lower_bound[task]
        previous_task_lower_bound = self.calculate_lower_bound(task - 1)
        min_next_task = min(self.computation_cost[task][previous_task_lower_bound[1]], self.communication_cost[task - 1][task] + np.min(self.computation_cost[task]))
        if min_next_task == self.computation_cost[task][previous_task_lower_bound[1]]:
            min_processor = previous_task_lower_bound[1]
        else:
            min_processor = np.argmin(self.computation_cost[task])
        self.task_lower_bound[task] = (previous_task_lower_bound[0] + min_next_task, min_processor)
        return self.task_lower_bound[task]

    def get_successor(self, task):
        return list(np.where(self.edges[task] == 1)[0])

    def get_predecessor(self, task):
        return list(np.where(np.transpose(self.edges)[task] == 1)[0])

    def get_average_computation_time(self, task):
        avg = 0
        count = 0
        for wcet in self.computation_cost[task]:
            if wcet != INFINITY:
                avg = (avg * count + wcet) / (count + 1)
                count += 1
        return avg

    def calculate_rank(self, task):
        if self.upward_rank[task] != 0:
            return self.upward_rank[task]

        _max = self.find_max_successor_computation_time(task)

        self.upward_rank[task] = self.get_average_computation_time(task) + _max
        return self.upward_rank[task]

    def find_max_successor_computation_time(self, task):
        _max = 0
        for successor_task in self.get_successor(task):
            candidate_max = self.communication_cost[task][successor_task] + self.calculate_rank(successor_task)
            if candidate_max > _max:
                _max = candidate_max
        return _max

    @staticmethod
    def get_n_max(criticality_slack):
        return criticality_slack + 1

    def add_to_priority_queue(self, function_num):
        for task_num in range(len(self.upward_rank)):
            heap_push(self.task_priority_queue, (self.upward_rank[task_num], self.criticality, task_num, function_num))
            # self.task_priority_queue.put((self.upward_rank[task_num], self.criticality, task_num, function_num))

In [5]:
class MS:
    def __init__(self, functions: list, criticality):
        self.functions = functions
        self.common_ready_queue = []
        self.criticality = criticality

        function_num = 0
        for function in functions:
            function.calculate_rank(0)
            function.add_to_priority_queue(function_num)
            function.calculate_lower_bound(len(function.upward_rank) - 1)
            function_num += 1

In [6]:
processor_1 = Processor()
processor_2 = Processor()
processor_3 = Processor()
processors_list = [processor_1, processor_2, processor_3]

communication_cost_1 = np.array([[0, 11, 8],
                                [11, 0, 6],
                                [8, 6, 0]])
computation_cost_1 = np.array([[INFINITY, 6, 11],
                              [17, 12, 6],
                              [13, 9, 10]])
edges_1 = np.array([[0, 1, 1],
                    [0, 0, 1],
                    [0, 0, 0]])

function_1 = Function(processors_list, n=3, criticality=3, communication_cost=communication_cost_1,
                      computation_time=computation_cost_1, edges=edges_1, arrival_time=0, lower_bound=27, deadline=34)

communication_cost_2 = np.array([[0, 3, 7],
                                [3, 0, 2],
                                [7, 2, 0]])
computation_cost_2 = np.array([[12, 18, 9],
                              [13, 10, INFINITY],
                              [INFINITY, 18, 7]])
edges_2 = np.array([[0, 1, 1],
                    [0, 0, 1],
                    [0, 0, 0]])

function_2 = Function(processors_list, n=3, criticality=1, communication_cost=communication_cost_2,
                      computation_time=computation_cost_2, edges=edges_2, arrival_time=10, lower_bound=31, deadline=46)

functions_list = [function_1, function_2]
ms = MS(functions=functions_list, criticality=0)

In [7]:
print(function_1.upward_rank)
print(function_2.upward_rank)


# print(function_1.communication_cost[0][1])
# print(function_1.task_lower_bound)
# print(function_2.task_lower_bound)
# print(function_2.get_task_abs_deadline(0))
# print(function_1.get_predecessor(2))
# np.min(function_1.computation_cost[1])
# while function_1.task_priority_queue:
#     print(heap_pop(function_1.task_priority_queue))

[47.83333333 28.33333333 10.66666667]
[42.  26.  12.5]


In [8]:
def task_priority_queues_is_empty(task_priority_queues: list):
    for task_priority_queue in task_priority_queues:
        if len(task_priority_queue) != 0:
            return False
    return True

In [9]:
task_priority_queues = [function_obj.task_priority_queue for function_obj in functions_list]
task_priority_queues

[[(47.83333333333333, 3, 0, 0),
  (28.33333333333333, 3, 1, 0),
  (10.666666666666666, 3, 2, 0)],
 [(42.0, 1, 0, 1), (26.0, 1, 1, 1), (12.5, 1, 2, 1)]]

In [43]:
while not task_priority_queues_is_empty(task_priority_queues):
    for function_obj in ms.functions:
        if function_obj.criticality < ms.criticality:
            continue
        criticality_slack = function_obj.criticality - ms.criticality
        n_max = Function.get_n_max(criticality_slack)
        cnt = n_max
        while cnt != 0 and len(function_obj.task_priority_queue) != 0:
            cnt -= 1
            if len(function_obj.task_priority_queue) == 0:
                break
            tuple_record = heap_pop(function_obj.task_priority_queue)
            print(f'pop task priority queue: {tuple_record}')
            heap_push(ms.common_ready_queue, tuple_record)
            print(f'ms.common_ready_queue: {ms.common_ready_queue}')

    while len(ms.common_ready_queue) != 0:
        tuple_record = heap_pop(ms.common_ready_queue)



AttributeError: 'Function' object has no attribute 'f'